In [3]:
!pip install transformers
!pip install yfinance
!pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 6.6 MB/s 
     |████████████████████████████████| 7.6 MB 60.9 MB/s 
     |████████████████████████████████| 182 kB 44.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.6 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import requests
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import datetime
import yfinance as yf
import pandas as pd
import pytz
import plotly.graph_objects as go

In [ ]:
API_TOKEN= 
#API_TOKEN options
#API_TOKEN = 'ceep5d2ad3i92ceb1ingceep5d2ad3i92ceb1io0'
#API_TOKEN = 'ceep5d2ad3i92ceb1ingceep5d2ad3i92ceb1io0'
#API_TOKEN = 'ceep5d2ad3i92ceb1ingceep5d2ad3i92ceb1io0'

In [15]:
tokenizer = AutoTokenizer.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

def fetch_prive_n_news(stock_id, start_date, end_date):
  from datetime import datetime
  timegroup=[datetime.strptime('14:30:00', '%H:%M:%S').time(),datetime.strptime('15:30:00', '%H:%M:%S').time(),datetime.strptime('16:30:00', '%H:%M:%S').time(),datetime.strptime('17:30:00', '%H:%M:%S').time(),datetime.strptime('18:30:00', '%H:%M:%S').time(),datetime.strptime('19:30:00', '%H:%M:%S').time(),datetime.strptime('20:30:00', '%H:%M:%S').time()]
  import datetime
  raw_df =  (
        yf.Ticker(stock_id)
        .history(start=start_date, end=end_date, interval='1h')
        .tz_convert(datetime.timezone.utc)
    )
  
  raw_df['timestamp'] = raw_df.index.astype('int64') // 10**9
  output = pd.DataFrame(requests.get(f'https://finnhub.io/api/v1/company-news?symbol={stock_id}&from={start_date}&to={end_date}&token={API_TOKEN}').json())
  output['datetime'] = output['datetime'].apply(lambda x: datetime.datetime.fromtimestamp(x))
  output['sentiment'] = output['summary'].apply(lambda x: classifier(x))
  output['sentiment_group'] = output['sentiment'].apply(lambda x: x[0]['label'])
  output['sentiment_score'] = output['sentiment'].apply(lambda x: x[0]['score'])
  # datetime values in your column to the nearest half past hour.
  output['datetime_group'] = output['datetime']
  output['time_group'] = output['datetime_group'].apply(lambda x:datetime.datetime.time(x))
  output['time_group'] = output['time_group'].apply(lambda x: timegroup[0] if x<=timegroup[0] else timegroup[1] if x<=timegroup[1] else timegroup[2] if x<=timegroup[2] else timegroup[3] if x<=timegroup[3] else timegroup[4] if x<=timegroup[4] else timegroup[5] if x<=timegroup[5] else timegroup[5] if x<=timegroup[6] else x)
  output['date'] = output['datetime_group'].apply(lambda x:datetime.datetime.date(x))
  output['datetime_group']=pd.to_datetime(output.date.astype(str) + ' ' + output.time_group.astype(str))
  output['datetime_group'] = output['datetime_group'].apply(lambda x:datetime.datetime.timestamp(x))
  sentiment_results = output.groupby(['datetime_group','sentiment_group'])['category'].agg('count').reset_index()
  df_dict = dict(zip(sentiment_results['datetime_group'], sentiment_results['sentiment_group'].apply(lambda x: -1 if x == 'negative' else 1 if x == 'positive' else 0) * sentiment_results['category']))
  raw_df = raw_df.merge(pd.DataFrame.from_dict(df_dict, orient='index', columns=['sentiment_score']), left_on='timestamp', right_index=True, how='left').fillna(0)

  #Draw a price Chart and draw a second y axis for sentiment score
  #fig = go.Figure()
  #fig.add_trace(go.Scatter(x=raw_df.index, y=raw_df['Close'], name='Price'))
  #fig.add_trace(go.Bar(x=raw_df.index, y=raw_df['sentiment_score'], name='Sentiment Score', yaxis='y2', opacity=0.5))
  #fig.update_layout(yaxis2=dict(overlaying='y', side='right'))
  #fig.show()
  return raw_df

In [ ]:
'one year news data obtaining'
import datetime as dt
end_date=dt.date.today()
result=pd.DataFrame()
for i in range(5,360,5):
 
    start_date= dt.date.today()- dt.timedelta(days = i)
    raw_df = fetch_prive_n_news('META', start_date,end_date)
    result=result.append(raw_df)
    end_date=dt.date.today()-dt.timedelta(days = i+1)

 #Draw a price Chart and draw a second y axis for sentiment score
 

In [22]:
result

,Open,High,Low,Close,Volume,Dividends,Stock Splits,timestamp,sentiment_score
2022-12-13 14:30:00+00:00,121.099998,123.300003,120.320000,121.297401,11910078,0,0,1670941800,1.0
2022-12-13 15:30:00+00:00,121.330002,122.480003,119.644997,121.468201,9461725,0,0,1670945400,0.0
2022-12-13 16:30:00+00:00,121.459999,122.199997,119.509003,119.510002,5322285,0,0,1670949000,0.0
2022-12-13 17:30:00+00:00,119.510002,119.989998,118.639999,119.910004,3309137,0,0,1670952600,0.0
2022-12-13 18:30:00+00:00,119.919998,120.290001,119.150002,120.279999,2622011,0,0,1670956200,0.0
...,...,...,...,...,...,...,...,...,...
2022-05-25 15:30:00+00:00,182.600006,183.199997,181.483002,181.859894,2143535,0,0,1653492600,0.0
2022-05-25 16:30:00+00:00,181.871307,182.729996,181.580002,181.910004,2300738,0,0,1653496200,0.0
2022-05-25 17:30:00+00:00,181.910004,183.649994,180.309998,183.449997,3981385,0,0,1653499800,0.0
2022-05-25 18:30:00+00:00,183.470001,185.449997,182.889999,184.490005,3584124,0,0,1653503400,0.0


In [33]:
 fig = go.Figure()
 fig.add_trace(go.Scatter(x=result.index, y=result['Close'], name='Price'))
 fig.add_trace(go.Bar(x=result.index, y=result['sentiment_score'], name='Sentiment Score', yaxis='y2', opacity=0.5, marker={'color':'red'}))
 fig.update_layout(yaxis2=dict(overlaying='y', side='right'))
 fig.show()